# Notebook para generar el dataset

Utilizando la biblioteca Spotipy vamos a generar un csv, haciendo una búsqueda general por año para obtener todas las tracks que fueron publicadas. Reduciremos el universo de datos al país Argentina.
Luego, de cada una de las tracks obtenidas se vovlerá a realizar consultas a la Web API de Spotify para obtener información un poco más técnica de cada una de las tracks.
Con todos esos datos, se generará un DataSet y se lo almacenará en un archivo CSV para su uso posterior.

In [2]:
import spotipy #pip install spotipy --upgrade
from spotipy.oauth2 import SpotifyClientCredentials#para gestionar la autenticacion contra la API de Spotify
import pandas as pd
import numpy as np

# Inicializamos Spotipy

##### Credenciales para conectarse a la web api de Spotify

In [3]:
SPOTIPY_CLIENT_ID = 'e8b2379bdd054eb993035179ae40e066'
SPOTIPY_CLIENT_SECRET = 'f191a3a661cc47d68b92a296669dde05'#este valor puede ser cambiado desde la app

##### Incializamos el objeto sp para poder realizar las requests

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager= client_credentials_manager)

# Realizamos las consultas por año

##### Seteamos los años en que las tracks fueron publicadas

In [21]:
#years = list(range(1990, 2020))#excluye el final
years = [1990]

##### Seteamos la cantidad de máxima de tracks que queremos tomar como muestra para cada uno de los años

In [53]:
max_cantidad_tracks_anio = 33

##### Realizamos la consulta

In [55]:
#declaramos un array de diccionarios donde vamos a ir poniendo los resultados, para luego generar el data frame
data = []

#loopeamos por los años seteados
for year in years:
    print('Año: ' + str(year))    
    
    #configuramos la consulta para el año actual
    search_query = 'year:' + str(year)
    
    #incializamos un contador de tracks, para no traer mas de las que configuramos en max_cantidad_tracks_anio
    tracks_obtenidas = 0        
    
    #definimos una cantidad maxima de reintentos ante errores
    max_retry = 3
    
    #la web api solo permite obtener hasta 50 items a la vez, es por eso que para llegar a cantidad max de tracks
    #por año, se deberá realizar más de una consulta para el mismo año
    while tracks_obtenidas < max_cantidad_tracks_anio:                
        
        #si ya no quedan reintentos, se abandona el año
        if(max_retry<=0):
            print('\nSe consumieron todos los reintentos, se abandona el año.')
            break
        
        try:
            #cuando se hace una consulta a la web api, spotify encuentra por ejemplo 120 resultados
            #pero solo permite traer de a 50 (limit) a la vez, para poder traer todos hay que ir moviéndose en ese
            #resultado con el offset. Para traer las primeras 50, el offset = 0, para traer las siguientes 50, el offset = 50
            result = sp.search(search_query, limit=50, offset=tracks_obtenidas, type='track', market='AR')

            if(result is not None and result['tracks'] is not None and len(result['tracks']['items']) > 0):
                #nos quedamos con las tracks
                tracks_results = list(result['tracks']['items'])

                #por cada uno de los tracks obtenidos, consultamos los datos tecnicos
                #y luego generamos una entrada en el array de diccionarios
                for track_result in tracks_results:
                    track_spotify_id = track_result['id']
                    album = track_result['album']
                    artist = track_result['artists'][0]
                    
                    #nos aseguramos de no agregar mas tracks que las que se establecieron en max_cantidad_tracks_anio
                    if(tracks_obtenidas < max_cantidad_tracks_anio):
                        #consultamos a la web api de spotify los datos tecnicos de la track actual                    
                        track_features_result = sp.audio_features(track_spotify_id)

                        if(track_features_result is not None and len(track_features_result) > 0):
                            track_features = track_features_result[0]                    

                            #agregamos los datos al array de diccionarios
                            data.append({'popularity': track_result['popularity'], 'id':track_spotify_id, 
                                         'name':track_result['name'], 'album': album['name'],
                                         'album_release_date': album['release_date'], 'artist':artist['name'],
                                         'danceability': track_features['danceability'], 'energy': track_features['energy'],
                                         'key': track_features['key'], 'loudness': track_features['loudness'],
                                         'mode': track_features['mode'], 'speechiness': track_features['speechiness'],
                                         'acousticness': track_features['acousticness'], 'instrumentalness': track_features['instrumentalness'],
                                         'liveness': track_features['liveness'], 'valence': track_features['valence'],
                                         'tempo': track_features['tempo']
                                        })

                            tracks_obtenidas += 1
                            print(str(tracks_obtenidas) + ' tracks obtenidas!', end='\r')

            else:
                print('Datos vacíos, reintentando')
                #ante cualquier error, bajamos la cantidad de reintentos y seguimos
                max_retry -= 1                    
        
        except Exception as e: 
            print('Error: '+ str(e))
            print('Reintentando')
            #ante cualquier error, bajamos la cantidad de reintentos y seguimos
            max_retry -=1
    print('\nAño completado')
            

Año: 1990
33 tracks obtenidas!
Año completado


# Generamos el DataFrame

In [56]:
df_tracks = pd.DataFrame(data)

In [59]:
df_tracks.shape

(33, 17)

In [69]:
df_tracks.head()

,popularity,id,name,album,album_release_date,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,58,6qoHBab3uXaz0qCXm04iTQ,Hacelo por Mí,El Cielo Puede Esperar,1990,Attaque 77,0.592,0.510,4,-12.316,1,0.0295,0.005190,0.002880,0.202,0.431,143.531
1,71,2lpIh6Gr6HYjg1CFBaucS5,De Música Ligera - Remasterizado 2007,Canción Animal (Remastered),1990-10-09,Soda Stereo,0.536,0.749,2,-6.392,1,0.0282,0.000638,0.555000,0.272,0.796,124.235
2,75,2SbsvihBJmgNKZ4qkXFWrp,More Than Words,Extreme II - Pornograffitti,1990-01-01,Extreme,0.623,0.127,6,-14.935,1,0.0306,0.452000,0.000000,0.114,0.237,91.625
3,81,57bgtoPSgt236HzfBOd8kj,Thunderstruck,The Razors Edge,1990-09-24,AC/DC,0.501,0.889,4,-5.175,1,0.0364,0.000147,0.011400,0.217,0.257,133.519
4,69,5kxol5m6MfiJgahqyzbDRH,Tan enamorados,Ricardo Montaner Con La London Metropolitan Or...,1990-01-01,Ricardo Montaner,0.413,0.530,3,-7.389,1,0.0326,0.472000,0.000001,0.158,0.149,138.680


# Generamos el CSV

In [61]:
tracks_csv_name = 'tracks.csv'

In [62]:
df_tracks.to_csv(tracks_csv_name,index=False,encoding='utf-8')